In [101]:
import pandas as pd
import re
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import unicodedata

##function to remove accents
def strip_accents(text):
    text=text.lower()
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text)

## Getting Soup
def scraping(input_team):
    url='https://dondeverlo.com/futbol/equipo/{}'.format(team)
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html,'html.parser')
    return soup

##next_match
def next_match(soup):
    matches=soup.findAll('h2',{"class":'Event__title'})
    match=matches[0].text
    match=match.lstrip().rstrip()
    return match

##team2
def team2(soup,team):
    matches=soup.findAll('h2',{"class":'Event__title'})
    match=matches[0].text
    match=match.lstrip().rstrip()
    
    nm2=re.sub("\s\s+" , " ",match)
    nm2=strip_accents(nm2)
    nm3=nm2.split(" - ",)
    local=nm3[0]
    visitor=nm3[1]
    if local==team:
        return visitor
    else:
        return local

##date_next_match
def date_next_match(soup):
    dates = soup.findAll('div',{"class":'Event__when'})
    date_match = dates[0].text
    date_match = date_match.lstrip().rstrip()
    return date_match

##competition_next_match
def competition_next_match(soup):
    competition = soup.findAll('span',{"class":'Event__championshipText'})
    competition_match = competition[0].text
    competition_match = competition_match.lstrip().rstrip()
    competition=competition_match.split("-")
    return competition[1].lstrip().rstrip()

##matchday_next_match
def matchday_next_match(soup):
    matchday = soup.findAll('div',{"class":'Event__description'})
    matchday_match = matchday[0].text
    matchday_match = matchday_match.lstrip().rstrip()
    return matchday_match

##summary scrapping next match
def printer(soup,team):
    return "El proximo partido de tu equipo es el {} contra {} correspondiente a la {} de {}.".format(date_match,contender,matchday_match,competition_match)

In [102]:
input_team="alavés"
team=strip_accents(input_team)
match=next_match(soup)
team2=team2(soup,team)
contender=strip_accents(team2)
date_match=date_next_match(soup)
competition_match=competition_next_match(soup)
matchday_match=matchday_next_match(soup)
                        
print(input_team)
print(team)
print(contender)
print(match)

print("Tu equipo: {}".format(team.upper()))
print("Tu rival: {}".format(contender.upper()))
print(printer(soup,team))

alavés
alaves
eibar
Eibar - Alavés
Tu equipo: ALAVES
Tu rival: EIBAR
El proximo partido de tu equipo es el domingo 24 de nov. 16:00 contra eibar correspondiente a la Jornada 14 de LaLiga.


In [103]:
def dfFilter(team,contender):
    df=pd.read_csv("./input/FMEL_Dataset.csv")
    df['localTeam']=df['localTeam'].map(lambda x: x.lower() if isinstance(x,str) else x)
    df['visitorTeam']=df['visitorTeam'].map(lambda x: x.lower() if isinstance(x,str) else x)
    dff=df[(df['timestamp']>=1093644000) & ((df['localTeam']==(team)) | (df['localTeam']==(contender))) & ((df['visitorTeam']==(contender)) | (df['visitorTeam']==(team)))]
    return dff.sort_values('timestamp', ascending=False)

##function to translate if your team wins, loses, or draws for each game
def result (row,team):
    if row['localGoals']==row['visitorGoals']:
        return 1
    if row['localTeam']==team and row['localGoals']>row['visitorGoals']:
        return 3
    if row['visitorTeam']==team and row['visitorGoals']>row['localGoals']:
        return 3
    else:
        return 0

##creating and appending new column
def summary(a,team,team2):
    a = a.value_counts()
    ##sign=a.value_counts()
    ##sign=df_team['result'].value_counts()
    return "{} y {} se han enfrentado {} veces en los últimos 20 años. El {} ha ganado {} partidos, perdido {}, y empatado {}.".format(team.upper(),team2.upper(),a.sum(),team.upper(),a[3],a[0],a[1])

In [105]:
df_filtered=dfFilter(team,contender)
df_filtered["result"] = df_filtered.apply(lambda df_filtered: result(df_filtered,team), axis=1)
Last_Games=df_filtered.head(5)

print(df_filtered.head())
print(Last_Games)

          id   season  division  round localTeam visitorTeam  localGoals  \
36623  36624  2017-18         1     32     eibar      alaves           0   
36429  36430  2017-18         1     13    alaves       eibar           1   
35800  35801  2016-17         1     34    alaves       eibar           0   
35608  35609  2016-17         1     15     eibar      alaves           0   
33755  33756  2013-14         2     40     eibar      alaves           1   

       visitorGoals        date   timestamp  result  
36623             1  15/04/2018  1523743200       3  
36429             2  25/11/2017  1511564400       0  
35800             0  27/04/2017  1493244000       1  
35608             0  11/12/2016  1481410800       1  
33755             0  25/05/2014  1400968800       0  
          id   season  division  round localTeam visitorTeam  localGoals  \
36623  36624  2017-18         1     32     eibar      alaves           0   
36429  36430  2017-18         1     13    alaves       eibar       